<a href="https://colab.research.google.com/github/ixd-ai-hub/Research-Ground/blob/project%2FCU-865d7n3wb-nft-generation/Image_Generator_V2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from random import choice, sample

def prompt_styler(prompts):
  # with open(style_config, 'r') as pf:
  #   config = json.load(pf)
  # print(config)
  config = {
    "main": [
      "a painting of",
      "a digital art of",
      "a digital painting of"
    ],
    "artist": [
      "by Charlie Bowater",
      "by Bastien Lecouffe-Deharme",
      "by Yoshitaka Amano",
      "by Karol Bak ",
      "by Yoann Lossel",
      "by Peter Mohrbacher",
      "by Ryohei Hase",
      "by Lü Ji",
      "by tooth wu",
      "by greg rutkowski",
      "by gaston bussiere",

      "by yoshitaka amano",
      "by tsutomu nihei",
      "by donato giancola",
      "by tim hildebrandt"
    ],
    "style": [
      "Cgsociety",
      "Poster art",
      "Digital painting",
      "Steampunk",
      "Gothic art",
      "Dystopian art",
      "Fantasy art",
      "Matte painting",
      "Cinematic lighting",
      "focus , sharp",
      "huge scene",
      "unreal 5",
      "hyper realism",
      "ultra detailed fantasy",
      "lotr game design fanart by concept art",
      "devinart",
      "concept art",
      "fantasy character portrait",
      "extreme detail"
    ]
  }
  # filter any random artist
  prompts = [p[:p.find('by')] for p in prompts]

  new_prompts = [
      "{main} {i}, {artist}, {style}, detailed, pixiv".format(main=choice(config['main']), i=i, artist=', '.join(sample(config['artist'], 3)), style=', '.join(sample(config['style'], 3)))
      for i in prompts
  ]
  return new_prompts

In [ ]:
from IPython import get_ipython
!pip install git+https://github.com/sberbank-ai/Real-ESRGAN.git
!wget https://huggingface.co/datasets/db88/Enhanced_ESRGAN/resolve/main/RealESRGAN_x4.pth -O /content/RealESRGAN_x4.pth
# @title <font color="cyan">Generate NFT</font>

DevMode = True
Version = "v0.6.11"
Model = "naclbit/trinart_stable_diffusion_v2"
#  # @param ["runwayml/stable-diffusion-v1-5", "stabilityai/stable-diffusion-2-1", "CompVis/stable-diffusion-v1-4", "naclbit/trinart_stable_diffusion_v2", "hakurei/waifu-diffusion"]

import importlib, os, shutil
try:
    importlib.reload(colab)
    importlib.reload(text2img)
except: pass
# Clone repo
if os.path.exists("/content/"):
    # In colab
    if os.path.exists("/content/void-diffusion"):
        shutil.rmtree('/content/void-diffusion')
    os.chdir("/content")
    if DevMode:
      print("Cloning an unstable version -> ", end="")
      get_ipython().system("git clone https://github.com/nipdep/void-diffusion.git &> /dev/null")
    else:
      print("Updating from nipdep/void-diffusion (%s) -> " % Version, end="")
      get_ipython().system("git clone --depth 1 --branch %s https://github.com/nipdep/void-diffusion.git &> /dev/null" % Version)
    print("Done.")
    try: os.chdir("/content/void-diffusion")
    except: print("Couldn't clone the repository. You sure that version exists?")
    print("Installing dependencies -> ", end="")
    !pip install einops torch transformers diffusers accelerate > /dev/null
    print("Done.")
    import colab
    importlib.reload(colab)
    try: colab.init(Model)
    except Exception as e:
      print("Error: ")
      print(e)
      print("\nThings you can try:\n- Just click again\n- Delete and restart the runtime\n- Select a different model")
else:
    # Not in colab
    if not os.path.basename(os.getcwd()) == "void-diffusion":
      if DevMode:
        print("Cloning an unstable version -> ", end="")
        !git clone https://github.com/nipdep/void-diffusion.git &> /dev/null
      else:
        print("Updating from nipdep/void-diffusion (%s) -> " % Version, end="")
        !git clone --depth 1 --branch v0.6 https://github.com/nipdep/void-diffusion.git &> /dev/null
      print("Done.")
      os.chdir("void-diffusion")
    !git pull >nul
    print("Installing dependencies -> ", end="")
    !pip install einops torch transformers diffusers accelerate >nul
    print("Done.")
    import colab
    try: colab.init(Model)
    except Exception as e:
      print("Error: ")
      print(e)
      print("\nThings you can try:\n- Just click again\n- Delete and restart the runtime\n- Select a different model")


try:
  import colab, os, shutil, datetime
  from IPython import display
  from IPython.display import HTML
  if colab.ready: # needed to trigger an exception
    #@title <font color="orange">Settings & Google Drive</font>
    #@markdown General
    Seed = 0 #@param {type:"number"}
    if Seed == None:
        Seed = 0
    Width = "2048" #@param [256, 512, 1024, 2048]
    Height = "2048" #@param [256, 512, 1024, 2048]
    Scheduler = "K-EULER" #param ["Default", "K-EULER", "DDIM", "K-LMS", "DPMSolver-Multistep"]
    #@markdown Google Drive settings
    SaveToGoogleDrive = False #@param {type:"boolean"}
    SaveDiffusionSettings = True # param {type:"boolean"}
    Directory = "AI-Gen" #@param {type:"string"}
    colab.settings['Seed'] = int(Seed)
    colab.settings['Width'] = int(Width)//4
    colab.settings['Height'] = int(Height)//4
    colab.settings['Scheduler'] = Scheduler
    colab.save_directory = Directory
    colab.save_settings = SaveDiffusionSettings
    ShouldSaveToGoogleDrive = False
    try:
      # GDrive
      if SaveToGoogleDrive:
        from google.colab import drive
        drive.mount('/content/gdrive', force_remount=True)
        ShouldSaveToGoogleDrive = True
      # Log
      display.display(HTML("<strong><span style='color: green'>Last saved at %s</span></strong>" % datetime.datetime.now().strftime("%H:%M")))
    except Exception as e: print(e)
except: print("Couldn't save settings! No model was selected.")


try:
  import colab, text2img, importlib, styler
  if colab.ready:
    importlib.reload(text2img)
    #@markdown <b>Prompt & Generator settings.</b>
    Prompt = "beautiful girl with a flower" #@param {type:"string"}
    Apply_Styling_to_the_Prompt = True #@param {type:"boolean"}
    if Apply_Styling_to_the_Prompt:
      Prompt = prompt_styler([Prompt])[0]

    NegativePrompt = "" #param {type:"string"}
    GuidanceScale = 13.8 #param {type:"slider", min:1, max:30, step:0.1}
    Steps = 100 #@param {type:"slider", min:1, max:500, step:1}
    Number_of_images = 100 #@param {type:"integer"}
    Preview = False #param {type:"boolean"}
    colab.settings["Prompt"] = Prompt
    colab.settings["NegativePrompt"] = NegativePrompt
    colab.settings['GuidanceScale'] = float(GuidanceScale)
    colab.settings["Steps"] = Steps
    colab.settings["Iterations"] = Number_of_images
    try:
      text2img.process(ShouldSave=ShouldSaveToGoogleDrive, ShouldPreview=Preview)
    except Exception as e: print(e)
except Exception as e: print(e) #: print("No model was selected.")


